In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from IPython.display import Javascript, display, clear_output, HTML

display(HTML("<style>.container { width:100% !important;}</style>"))

import pandas as pd

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [2]:
import os, requests
from typing import Dict
from pathlib import Path

In [3]:
repo_path = '/Users/thomasrichardson/Documents/GitHub/predict_the_prem/predict_the_prem/'

In [4]:
base_url = "https://fantasy.premierleague.com/api/"

In [4]:
def get_fixtures_df() -> pd.DataFrame:
    renaming_dict = {
        "code": "global_match_id",
        "id": "match_id",
        "event": "game_week",
        "finished": "is_finished",
        "finished_provisional": "is_finished_provisional",
        "kickoff_time": "match_start_time",
        "provisional_start_time": "is_provisional_start_time",
        "started": "is_started",
        "team_h": "home_team_id",
        "team_a": "away_team_id",
        "team_h_score": "home_team_score",
        "team_a_score": "away_team_score",
        "minutes": "minutes_played",
        "stats": "stats",
        "team_h_difficulty": "home_team_difficulty",
        "team_a_difficulty": "away_team_difficulty",
        "pulse_id": "pulse_id",
    }
    fixtures_url = os.path.join(base_url, "fixtures/")
    fixtures_response = requests.get(fixtures_url)
    fixtures_df = pd.DataFrame(fixtures_response.json())
    fixtures_df = fixtures_df.rename(columns=renaming_dict)[list(renaming_dict.values())]
    fixtures_df["match_start_time"] = pd.to_datetime(fixtures_df["match_start_time"])
    return fixtures_df

In [8]:
def get_teams_df() -> pd.DataFrame:
    renaming_dict = {
        "code": "global_team_id",
        "id": "team_id",
        "name": "team_name",
        "short_name": "team_short_name",
        "position": "league_position",
        "strength": "team_strength",
        "strength_overall_home": "team_strength_overall_home",
        "strength_overall_away": "team_strength_overall_away",
        "strength_attack_home": "team_strength_attack_home",
        "strength_attack_away": "team_strength_attack_away",
        "strength_defence_home": "team_strength_defence_home",
        "strength_defence_away": "team_strength_defence_away",
        "pulse_id": "pulse_id",
    }
    bootstrap_url = os.path.join(base_url, "bootstrap-static/")
    bootstrap_response = requests.get(bootstrap_url)
    teams_df = pd.DataFrame(bootstrap_response.json()["teams"])
    return teams_df.rename(columns=renaming_dict)[list(renaming_dict.values())]

In [13]:
def get_entries() -> Dict[str, pd.DataFrame]:
    directory = os.path.join(repo_path, "data/entries/")
    entries = {}
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"):
            new_entry_df = pd.read_csv(os.path.join(directory, filename))
            new_entry_name = "_".join(filename[:-4].split("_")[1:])
            entries[new_entry_name] = new_entry_df
    return dict(entries.items())

In [6]:
fixtures_df = get_fixtures_df()

In [9]:
teams_df = get_teams_df()

In [14]:
entries = get_entries()

In [5]:
bootstrap_url = os.path.join(base_url, "bootstrap-static/")
bootstrap_response = requests.get(bootstrap_url)

In [10]:
pd.DataFrame(bootstrap_response.json()["elements"]).sort_values("selected_rank_type")[["web_name", "selected_rank_type"]].head(20)

,web_name,selected_rank_type
385,M.Salah,1
248,João Pedro,1
619,Wan-Bissaka,1
139,Kelleher,1
0,Raya,2
505,Isak,2
374,Frimpong,2
234,Palmer,2
32,Gyökeres,3
508,Sels,3
